In [1]:
!pip install influxdb-client beautifulsoup4 loguru finmind pyarrow tqdm requests backtrader backtrader[plotting] shioaji && cp bug-fix/locator.py /opt/conda/lib/python3.10/site-packages/backtrader/plot/locator.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.2/705.2 kB 19.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 27.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.1/410.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 44.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9

In [ ]:
# compile ta-lib and install 
!tar -xzf ta-lib-0.4.0-src.tar.gz && \
cd ta-lib/ && \
./configure --prefix=/usr && \
make && \
sudo make install

!pip install finmind TA-Lib pyarrow tqdm requests backtrader backtrader[plotting] shioaji && cp bug-fix/locator.py /opt/conda/lib/python3.10/site-packages/backtrader/plot/locator.py

# Data Loader

In [ ]:
# configs
API_TOKEN=''
DATA_DIR='mtx-records'
START_DATE='3/1/2015'
END_DATE='7/21/2022'

In [ ]:
from time import sleep

import pandas as pd
from tqdm import tqdm
from FinMind.data import DataLoader

api = DataLoader()
api.login_by_token(api_token=API_TOKEN)

def download_mtx(date_):
    df = api.taiwan_futures_tick(
        futures_id='MTX',
        date=date_
    )
    return df

datelist = pd.date_range(start=START_DATE, end=END_DATE)

with open('progress.txt', 'w') as f:
    for d in tqdm(datelist):
        date_ = str(d.date())
        df = download_mtx(date_)        
        if df.size:
            df.to_feather(f'{DATA_DIR}/{date_}')
        
        f.write(f'{date_}\t{df.size}\n')
        f.flush()

        sleep(20)

In [44]:
from datetime import datetime, timedelta
import random
from secrets import choice

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
TOKEN = ''
ORG = 'my-org'
BUCKET = 'bb'
client = InfluxDBClient(url='http://localhost:8086', token=TOKEN)
write_api = client.write_api(write_options=SYNCHRONOUS)

In [64]:
import random

In [6]:
for _ in range(10000):
    point = (
        Point('hello')
            .tag('code', 'abc')
            .field('close', random.randint(1, 100))
            .time(datetime.now(), WritePrecision.NS)
    )

    write_api.write(BUCKET, ORG, point)

In [30]:
t1 = datetime(2022, 8, 5, 13, 43, 54, 734000)
t2 = datetime(2022, 8, 5, 13, 45, 54, 765)

In [49]:
def write(ts):
    point = (
        Point('hello')
            .tag('code', 'abc')
            .field('close', random.randint(1, 100))
            .time(ts, WritePrecision.NS)
    )
    print(point)
    print(BUCKET)
    print(ORG)
    write_api.write(BUCKET, ORG, point)

In [53]:
point = (
            Point('index') \
                .tag('code', 'abc') \
                .field('close', 100) \
                .time(datetime.utcnow(), WritePrecision.NS)
        )
r = write_api.write('bb', ORG, point)

In [51]:
write(datetime.utcnow())
write(t2)

hello,code=abc close=4i 1659933889815746000
bb
my-org
hello,code=abc close=64i 1659707154000765000
bb
my-org


In [32]:
def write_point(point: Point):
    write_api.write(BUCKET, ORG, point)

def create_point(host: str, used_percent: float, time: datetime):
    # yapf: disable
    return (
        Point('mem')
            .tag('host', host)
            .field('used_percent', used_percent)
            .time(time, WritePrecision.NS)
    )




def generate_fake_points():
    hosts = ['host-A', 'host-B', 'host-C']
    # date_list = [datetime.utcnow() - timedelta(minutes=x) for x in range(1000)]
    date_list = [datetime.now() - timedelta(minutes=x) for x in range(1000)]

    for i in range(10):
        point = create_point(random.choice(hosts), random.uniform(10, 50), date_list[999-i])
        point = create_point(hosts[1], random.uniform(10, 50), date_list[999-i])
        print(point)
        write_point(point)

BUCKET = 'now'
generate_fake_points()


mem,host=host-B used_percent=15.434020949483171 1659897793713913000


ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': '2.0.9', 'X-Platform-Error-Code': 'not found', 'Date': 'Mon, 08 Aug 2022 03:22:13 GMT', 'Content-Length': '57'})
HTTP response body: {"code":"not found","message":"bucket \"now\" not found"}
